# Test Atlas v2

See atlas v1 issues:
https://github.com/roocs/clisops/issues/317

In [1]:
import xarray as xr
from clisops.core import subset
from clisops.ops import subset as subset_op

import time
import os

In [2]:
basedir_atlas_v2 = "/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset"

cmip6_nc = f"{basedir_atlas_v2}/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc"
cmip6_nc

'/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc'

## ncdump - CMIP6

In [3]:
! du -sh /mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc

5.6G	/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc


In [4]:
! ncdump -sh /mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc

netcdf t_CMIP6_historical_mon_185001-201412_v02 {
dimensions:
	bnds = 2 ;
	lon = 360 ;
	lat = 180 ;
	time = 1980 ;
	member = 30 ;
variables:
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "latitude" ;
		lat:bounds = "lat_bnds" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_Fletcher32 = "true" ;
		lat:_Shuffle = "true" ;
		lat:_DeflateLevel = 1 ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_Fletcher32 = "true" ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_DeflateLevel = 1 ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "longitude" ;
		lon:bounds = "lon_bnds" ;
		lon:_Storage = "chunked" ;
		lon:_ChunkSizes = 360 ;
		lon:_Fletcher32 = "true" ;
		lon:_Shuffle = "true" ;
		lon:_DeflateLevel = 1

## xarray - CMIP6

TODO: still using two fill values for variable t. Without a fix it is not possible to write as netcdf file.

Error Message: 
Variable 't' has conflicting _FillValue (-1.7014118346046923e+38) and missing_value (1.0384593717069655e+34). Cannot encode data.

In [5]:
def ds_info(ds):
    # compression levels
    print("data vars with compression:")
    for var in ds.data_vars:
        # print(var, ds[var].encoding, "\n")
        complevel = ds[var].encoding.get("complevel", 0)
        print(var, "compression level =", complevel)
        
    # fill values
    print("\nfill values:")
    var_list = list(ds.coords) + list(ds.data_vars)
    for var in var_list:
        fill_value = ds[var].encoding.get("_FillValue")
        print(var, "fill value =", fill_value)
        
    # string attributes with compression
    print("\nstring attributes with compression:")
    for cvar in [
            "member_id",
            "gcm_variant",
            "gcm_model",
            "gcm_institution",
            "rcm_variant",
            "rcm_model",
            "rcm_institution",
        ]:
            for en in ["zlib", "shuffle", "complevel"]:
                try:
                    print(cvar, en, ds[cvar].encoding[en])
                except KeyError:
                    pass

In [6]:
ds_cmip6 = xr.open_dataset(cmip6_nc)
ds_cmip6

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:543: SerializationWarning: variable 't' has multiple fill values {1.0384594e+34, -1.7014118e+38}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 1980, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 1850-01-01 1850-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [7]:
ds_info(ds_cmip6)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = -1.7014118e+38
crs fill value = None

string attributes with compression:
member_id zlib True
member_id shuffle True
member_id complevel 1
gcm_variant zlib True
gcm_variant shuffle True
gcm_variant complevel 1
gcm_model zlib True
gcm_model shuffle True
gcm_model complevel 1
gcm_institution zlib True
gcm_institution shuffle True
gcm_institution complevel 1


In [8]:
ds = ds_cmip6.isel(time=slice(0, 249), lon=slice(30, 50), lat=slice(50, 70))
ds

<xarray.Dataset>
Dimensions:          (lat: 20, bnds: 2, lon: 20, time: 249, member: 30)
Coordinates:
  * lat              (lat) float64 -39.5 -38.5 -37.5 -36.5 ... -22.5 -21.5 -20.5
  * lon              (lon) float64 -149.5 -148.5 -147.5 ... -132.5 -131.5 -130.5
  * time             (time) datetime64[ns] 1850-01-01 1850-02-01 ... 1870-09-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [9]:
from rook.utils import atlas_fixes

try:
    ds = atlas_fixes.apply_atlas_fixes("c3s-atlas-v2.cmip6", ds)
    ds.to_netcdf("/tmp/atlas_v2_cmip6_4.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)

debug var lat {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (180,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (180,), 'dtype': dtype('float64')}
debug var lon {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (360,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (360,), 'dtype': dtype('float64')}
debug var time {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (1980,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dat

## clisops-core - cmip6

TODO: clisops-core has the same issues like xarray. It does not apply any fixes. 

In [10]:
ds = subset.subset_bbox(
    ds_cmip6, lat_bnds=[45, 50], lon_bnds=[-60, -55],
    start_date='2013-01', end_date='2013-12')
ds

<xarray.Dataset>
Dimensions:          (lat: 5, bnds: 2, lon: 5, time: 12, member: 30)
Coordinates:
  * lat              (lat) float64 45.5 46.5 47.5 48.5 49.5
  * lon              (lon) float64 -59.5 -58.5 -57.5 -56.5 -55.5
  * time             (time) datetime64[ns] 2013-01-01 2013-02-01 ... 2013-12-01
    member_id        (member) object 'AS-RCEC_TaiESM1_r1i1p1f1' ... 'NUIST_NE...
    gcm_institution  (member) object 'AS-RCEC' 'AWI' ... 'NOAA-GFDL' 'NUIST'
    gcm_model        (member) object 'TaiESM1' 'AWI-CM-1-1-MR' ... 'NESM3'
    gcm_variant      (member) object 'r1i1p1f1' 'r1i1p1f1' ... 'r1i1p1f1'
    height2m         float64 2.0
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2

In [11]:
try:
    ds = atlas_fixes.apply_atlas_fixes("c3s-atlas-v2.cmip6", ds)
    ds.to_netcdf("/tmp/atlas_v2_cmip6.nc")
except ValueError as e:
    print("Fails to write as netcdf!", e)

debug var lat {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (180,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (180,), 'dtype': dtype('float64')}
debug var lon {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (360,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (360,), 'dtype': dtype('float64')}
debug var time {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (1980,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dat

## clisops-ops - cmip6 - subset by time

TODO: subset operation takes *very* long ... about 11 secs. It should only take much less than a second!

In [12]:
# clean up outputs

! rm /tmp/output_*

In [13]:
start = time.time()

outputs = subset_op(
    ds=ds_cmip6,
    time="2000-01/",
    # area=(0.0, 49.0, 10.0, 65.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

print("Number of output files: ", len(outputs))
outputs[0]

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable member_id has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_institution has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_model has data in the form of a dask array with dtype=object, which means it is being loaded int

duration: 35.39861226081848 secs
Number of output files:  1


'/tmp/output_001.nc'

In [14]:
file_size = os.path.getsize(outputs[0])
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 529.0558881759644 MB


In [15]:
ds = xr.open_dataset(outputs[0])
ds

<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 180, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2

In [16]:
ds_info(ds)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = 1.0384594e+34
crs fill value = None

string attributes with compression:
member_id zlib False
member_id shuffle False
member_id complevel 0
gcm_variant zlib False
gcm_variant shuffle False
gcm_variant complevel 0
gcm_model zlib False
gcm_model shuffle False
gcm_model complevel 0
gcm_institution zlib False
gcm_institution shuffle False
gcm_institution complevel 0


In [17]:
ds.to_netcdf("/tmp/atlas_v2_cmip6.nc")

In [18]:
! ncdump -sh /tmp/atlas_v2_cmip6.nc

netcdf atlas_v2_cmip6 {
dimensions:
	lat = 180 ;
	bnds = 2 ;
	lon = 360 ;
	time = 180 ;
	member = 30 ;
variables:
	double lat(lat) ;
		lat:_FillValue = NaN ;
		lat:standard_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
		lat:long_name = "latitude" ;
		lat:bounds = "lat_bnds" ;
		lat:_Storage = "chunked" ;
		lat:_ChunkSizes = 180 ;
		lat:_Fletcher32 = "true" ;
		lat:_Shuffle = "true" ;
		lat:_DeflateLevel = 1 ;
		lat:_Endianness = "little" ;
	double lat_bnds(lat, bnds) ;
		lat_bnds:_FillValue = NaN ;
		lat_bnds:coordinates = "height2m" ;
		lat_bnds:_Storage = "chunked" ;
		lat_bnds:_ChunkSizes = 180, 2 ;
		lat_bnds:_Fletcher32 = "true" ;
		lat_bnds:_Shuffle = "true" ;
		lat_bnds:_DeflateLevel = 1 ;
		lat_bnds:_Endianness = "little" ;
	double lon(lon) ;
		lon:_FillValue = NaN ;
		lon:standard_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
		lon:long_name = "longitude" ;
		lon:bounds = "lon_bnds" ;
		lon:_Storage = "chunked" ;
		lon:_ChunkS

## clisops-ops - cmip6 - subset by bbox

TODO: subset bbox is not possible! It uses a lot of memory ... even my 32GB VM was not enough for a successful run.

In [19]:
if True:
    start = time.time()

    outputs = subset_op(
        ds=ds_cmip6,
        time="2013-01-01/2013-12-30",
        area=(0.0, 49.0, 10.0, 65.0),
        output_type="xarray",
    )

    duration = time.time() - start
    print(f"duration: {duration} secs")

    outputs[0]

start cf_convert_between_lon_frames
lon interval: (0.0, 10.0)
conv 0.1
detect 1
coord 1
coord 2
coord 3
coord 4 lat
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
type 3
coord 4 lon
type 1
islon 1
type 2
coord 4 time
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
type 5
coord 4 member_id
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 gcm_institution
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 gcm_model
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 gcm_variant
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 height2m
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
type 4
coord 4 lat_bnds
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
coord 4 lon_bnds
ty

## clisops-ops - cmip6 - subset by bbox ... with fix

In [20]:
ds = xr.open_dataset("/tmp/atlas_v2_cmip6.nc")
ds

<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 180, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2014-12-01
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2

In [21]:
from rook.utils import atlas_fixes

if False:
    ds = atlas_fixes.apply_atlas_fixes("c3s-atlas-v2.cmip6", ds_cmip6)
    ds

In [22]:
ds_info(ds)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = nan
lon fill value = nan
time fill value = nan
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = nan
lat_bnds fill value = nan
lon_bnds fill value = nan
time_bnds fill value = nan
t fill value = 1.0384594e+34
crs fill value = None

string attributes with compression:
member_id zlib False
member_id shuffle False
member_id complevel 0
gcm_variant zlib False
gcm_variant shuffle False
gcm_variant complevel 0
gcm_model zlib False
gcm_model shuffle False
gcm_model complevel 0
gcm_institution zlib False
gcm_institution shuffle False
gcm_institution complevel 0


In [23]:
# clean up outputs

! rm /tmp/output_*

In [24]:
start = time.time()

outputs = subset_op(
    ds=ds,
    time="2013-01/2013-01",
    area=(0.0, 49.0, 10.0, 65.0),
    output_type="nc",
    # output_type="xarray",
    output_dir="/tmp",
    split_method="time:auto",
    file_namer="simple"
)

duration = time.time() - start
print(f"duration: {duration} secs")

outputs[0]

start cf_convert_between_lon_frames
lon interval: (0.0, 10.0)
conv 0.1
detect 1
coord 1
coord 2
coord 3
coord 4 lat
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
type 3
coord 4 lon
type 1
islon 1
type 2
coord 4 time
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
type 5
coord 4 member_id
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 gcm_institution
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 gcm_model
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 gcm_variant
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
type 6
coord 4 height2m
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
type 4
coord 4 lat_bnds
type 1
islon 1
islon 2
islon 3
islon 4
islon 5
istime 1
istime 2
istime 3
istime 4
istime 5
istime 6
coord 4 lon_bnds
ty

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable member_id has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_institution has data in the form of a dask array with dtype=object, which means it is being loaded into memory to determine a data type that can be safely stored on disk. To avoid this, coerce this variable to a fixed-size dtype with astype() before saving it.
  warnings.warn(
/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:224: SerializationWarning: variable gcm_model has data in the form of a dask array with dtype=object, which means it is being loaded int

'/tmp/output_001.nc'

## work on fix ... chatgpt

In [25]:


def fix_atlas(file_path):
    """
    Fixes a CMIP6 dataset by uncompressing bounds (time_bnds, lat_bnds, lon_bnds)
    and decoding string attributes.
    
    Parameters:
        file_path (str): Path to the CMIP6 NetCDF file.
    
    Returns:
        xarray.Dataset: The fixed dataset.
    """
    # Open the dataset
    ds = xr.open_dataset(file_path, decode_times=False)

    def uncompress_bounds(ds, var_name):
        """Uncompresses bounds for a given variable."""
        if var_name in ds.variables:
            if 'bounds' in ds[var_name].attrs:
                bounds_name = ds[var_name].attrs['bounds']
                if bounds_name in ds.variables:
                    ds[bounds_name] = xr.decode_cf({bounds_name: ds[bounds_name]})[bounds_name]
        return ds

    def uncompress_string_attributes(ds):
        """Decodes string attributes stored as bytes."""
        for var_name, var in ds.variables.items():
            for attr_name, attr_value in var.attrs.items():
                if isinstance(attr_value, bytes):  # Check if the attribute is compressed as bytes
                    try:
                        var.attrs[attr_name] = attr_value.decode("utf-8")
                    except Exception as e:
                        print(f"Failed to decode attribute '{attr_name}' in variable '{var_name}': {e}")

        # Also check global attributes
        for attr_name, attr_value in ds.attrs.items():
            if isinstance(attr_value, bytes):  # Check if the attribute is compressed as bytes
                try:
                    ds.attrs[attr_name] = attr_value.decode("utf-8")
                except Exception as e:
                    print(f"Failed to decode global attribute '{attr_name}': {e}")

        return ds

    # Apply fixes
    if 'time' in ds.variables:
        # ds = uncompress_bounds(ds, 'time')
        pass
    if 'lat' in ds.variables:
        # ds = uncompress_bounds(ds, 'lat')
        pass
    if 'lon' in ds.variables:
        # ds = uncompress_bounds(ds, 'lon')
        pass

    ds = uncompress_string_attributes(ds)

    return ds


In [26]:
ds_fixed = fix_atlas(cmip6_nc)
ds_fixed

/home/k/k204199/.envs/rook/lib/python3.11/site-packages/xarray/conventions.py:543: SerializationWarning: variable 't' has multiple fill values {1.0384594e+34, -1.7014118e+38}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 1980, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) float64 0.0 31.0 59.0 ... 6.02e+04 6.023e+04
    member_id        (member) object ...
    gcm_institution  (member) object ...
    gcm_model        (member) object ...
    gcm_variant      (member) object ...
    height2m         float64 ...
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) float64 ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38

In [27]:
ds_info(ds_fixed)

data vars with compression:
lat_bnds compression level = 1
lon_bnds compression level = 1
time_bnds compression level = 1
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = -1.7014118e+38
crs fill value = None

string attributes with compression:
member_id zlib True
member_id shuffle True
member_id complevel 1
gcm_variant zlib True
gcm_variant shuffle True
gcm_variant complevel 1
gcm_model zlib True
gcm_model shuffle True
gcm_model complevel 1
gcm_institution zlib True
gcm_institution shuffle True
gcm_institution complevel 1


## work on fix ... self made

In [28]:
def fix_atlas_v2(ds):
    """
    See also clisops.ops.base_operaton._remove_redundant_fill_values
    """
    if isinstance(ds, xr.Dataset):
        var_list = list(ds.coords) + list(ds.data_vars)
    elif isinstance(ds, xr.DataArray):
        var_list = list(ds.coords)
    # DEBUG
    for var in var_list:
        print("debug var", var, ds[var].encoding)
    # DEBUG END
    for var in var_list:
        ds[var].encoding["_FillValue"] = None
    # Remove string deflation options if applicable
    for cvar in [
        "member_id",
        "gcm_variant",
        "gcm_model",
        "gcm_institution",
        "rcm_variant",
        "rcm_model",
        "rcm_institution",
    ]:
        for en in ["zlib", "shuffle", "complevel"]:
            try:
                del ds[cvar].encoding[en]
            except KeyError:
                pass
    # use compression level 1
    for var in ds.data_vars:
       complevel = ds[var].encoding.get("complevel", 0)
       if "bnds" in var:
           ds[var].encoding["complevel"] = 0
           ds[var].encoding["zlib"] = False
           ds[var].encoding["shuffle"] = False
    return ds


In [29]:
ds_fixed = fix_atlas_v2(ds_cmip6)
ds_fixed

debug var lat {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (180,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (180,), 'dtype': dtype('float64'), '_FillValue': None, 'coordinates': None}
debug var lon {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunksizes': (360,), 'source': '/mnt/lustre/work/ik1017/C3SATLAS_v2_test/c3s-atlas-dataset/CMIP6/historical/t_CMIP6_historical_mon_185001-201412_v02.nc', 'original_shape': (360,), 'dtype': dtype('float64'), '_FillValue': None, 'coordinates': None}
debug var time {'zlib': True, 'szip': False, 'zstd': False, 'bzip2': False, 'blosc': False, 'shuffle': True, 'complevel': 1, 'fletcher32': True, 'contiguous': False, 'chunk

<xarray.Dataset>
Dimensions:          (lat: 180, bnds: 2, lon: 360, time: 1980, member: 30)
Coordinates:
  * lat              (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon              (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * time             (time) datetime64[ns] 1850-01-01 1850-02-01 ... 2014-12-01
    member_id        (member) object 'AS-RCEC_TaiESM1_r1i1p1f1' ... 'NUIST_NE...
    gcm_institution  (member) object 'AS-RCEC' 'AWI' ... 'NOAA-GFDL' 'NUIST'
    gcm_model        (member) object 'TaiESM1' 'AWI-CM-1-1-MR' ... 'NESM3'
    gcm_variant      (member) object 'r1i1p1f1' 'r1i1p1f1' ... 'r1i1p1f1'
    height2m         float64 2.0
Dimensions without coordinates: bnds, member
Data variables:
    lat_bnds         (lat, bnds) float64 ...
    lon_bnds         (lon, bnds) float64 ...
    time_bnds        (time, bnds) datetime64[ns] ...
    t                (member, time, lat, lon) float32 ...
    crs              int32 ...
Attributes: (12/27)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         180.0
    geospatial_lon_resolution:  1.0
    geospatial_lon_units:       degrees_east
    date_created:               2024-12-05 15:51:48.916442+01:00
    tracking_id:                4f887b4c-eaff-448c-9e50-4cacd5d16f38
    project_id:                 c3s-atlas-v2

In [30]:
ds_info(ds_fixed)

data vars with compression:
lat_bnds compression level = 0
lon_bnds compression level = 0
time_bnds compression level = 0
t compression level = 1
crs compression level = 0

fill values:
lat fill value = None
lon fill value = None
time fill value = None
member_id fill value = None
gcm_institution fill value = None
gcm_model fill value = None
gcm_variant fill value = None
height2m fill value = None
lat_bnds fill value = None
lon_bnds fill value = None
time_bnds fill value = None
t fill value = None
crs fill value = None

string attributes with compression:


In [31]:
if False:
    start = time.time()

    outputs = subset_op(
        ds=ds_fixed,
        time="2013-01/2013-01",
        area=(0.0, 49.0, 10.0, 65.0),
        output_type="xarray",
    )

    duration = time.time() - start
    print(f"duration: {duration} secs")

    outputs[0]